In [71]:
from lxml import etree

In [104]:
%%time
patent = []
claims = ""
tags = ['publication-reference','application-reference','invention-title',\
        'us-applicant','inventors','abstract','claims']
doc_number = 0
mydict = dict()
for _, element in etree.iterparse('ip4.xml', tag=tags):
    if element.tag == 'publication-reference':
        if mydict:
            patent.append(mydict)
        mydict = dict()
        for elem in element.iter('doc-number','country','date'):
            if elem.tag == 'doc-number':
                mydict[elem.tag] = int(elem.text)
            else:
                mydict[elem.tag] = elem.text
        element.clear()
    elif element.tag == 'application-reference':
        subdict = dict()
        for elem in element.iter('doc-number','country','date'):
            if elem.tag == 'doc-number':
                subdict[elem.tag] = int(elem.text)
            else:
                subdict[elem.tag] = elem.text
            mydict['application-reference'] = subdict
        element.clear()
    elif element.tag == 'invention-title':
        mydict['title'] = element.text
        element.clear()
    elif element.tag == 'us-applicant':
        subdict = dict()
        for e in element.iter('orgname','city','state','country'):
            subdict[e.tag] = e.text
        mydict['assignee'] = subdict
        element.clear()
    elif element.tag == 'inventors':
        mydict['inventors'] = []
        for elem in element.iter('inventor'):
            subdict = dict()
            for e in elem.iter('first-name','last-name','city','state','country'):
                subdict[e.tag] = e.text
            mydict['inventors'].append(subdict)
        element.clear()
    elif element.tag == 'abstract':
        mydict['abstract'] = ''.join([text for text in element.itertext() if element.tag not in ('b', 'i', 'u')])
        element.clear()
    elif element.tag == 'claims':
        mydict['claims'] = ''.join([text for text in element.itertext('claim-text') if element.tag not in ('b', 'i', 'u')])
        element.clear()

patent.append(mydict) # append last one
#patent

CPU times: user 2.44 s, sys: 828 ms, total: 3.27 s
Wall time: 4.35 s


In [106]:
with open('patents.json', 'w') as json_file:
    json.dump(patent, json_file, indent=2) 

In [82]:
for _, element in etree.iterparse('ip1.xml', tag=tags):
    print(element.tag)

publication-reference
invention-title
us-applicant
inventors
abstract
claims


In [65]:
import json
json.dumps(patent)

'[{"country": "US", "doc-number": 20230225235, "title": "AGRICULTURAL TRENCH DEPTH SYSTEMS, METHODS, AND APPARATUS", "assignee": {"orgname": "Precision Planting LLC", "city": "Tremont", "state": "IL", "country": "US"}, "inventors": [{"last-name": "Schlipf", "first-name": "Ben L.", "city": "Morton", "state": "IL", "country": "US"}, {"last-name": "Schlipf", "first-name": "Ben L.", "city": "Morton", "state": "IL", "country": "US"}, {"last-name": "Schlipf", "first-name": "Ben L.", "city": "Morton", "state": "IL", "country": "US"}], "abstract": "\\nA row unit (10) of an agricultural planter with an apparatus for adjusting the depth of a trench opened by the row unit (10). The row unit (10) includes a trench depth adjustment assembly (90T) configured to modify the trench depth. The trench depth adjustment assembly (90T) includes a depth adjustment body (5044) pivotally connected via a pivot to a frame member (14) of the row unit (10). An electric motor (5030) is operable to cause rotation of

In [101]:
for event, element in etree.iterparse('xml_data.xml',events=['start','end']):
    element.get('c')


XMLSyntaxError: Opening and ending tag mismatch: c line 4 and root, line 6, column 8 (xml_data.xml, line 6)

In [102]:
from lxml import etree

xml_content = """
<root>
    <item>1</item>
    <item>2</item>
    <item>3</item>
    <skipped_item>4
    <item>5</item>
</root>
"""

# Function to check if the element has a closing tag
def has_closing_tag(element):
    return element.text and '</' + element.tag + '>' in element.tail

# Parse the XML content
root = etree.fromstring(xml_content)

# Iterate over elements using iterparse
for event, element in etree.iterparse(root, events=('start', 'end')):
    if event == 'start':
        # Process start events
        print(f"Start: {element.tag}")
    elif event == 'end':
        # Process end events
        print(f"End: {element.tag}")

        # Check if the element has a closing tag
        if not has_closing_tag(element):
            print(f"Skipping incomplete element: {element.tag}")
            element.clear()  # Clear the element to free memory
            continue

        # Your processing logic here for non-skipped tags

# Clean up the remaining memory used by the context
del root


XMLSyntaxError: Opening and ending tag mismatch: skipped_item line 6 and root, line 8, column 8 (<string>, line 8)

In [103]:
from lxml import etree, html

xml_content = """
<root>
    <item>1</item>
    <item>2</item>
    <item>3</item>
    <skipped_item>4
    <item>5</item>
</root>
"""

# Use the HTML parser with recovery mode to handle incomplete XML
root = html.fromstring(xml_content)

# Convert the HTML tree back to XML
repaired_xml = etree.tostring(root, method='xml', encoding='utf-8').decode('utf-8')

# Parse the repaired XML
repaired_root = etree.fromstring(repaired_xml)

# Iterate over elements using iterparse
for event, element in etree.iterparse(repaired_root, events=('start', 'end')):
    if event == 'start':
        print(f"Start: {element.tag}")
    elif event == 'end':
        print(f"End: {element.tag}")

# Clean up the remaining memory used by the context
del repaired_root

TypeError: expected str, bytes or os.PathLike object, not _Element